# **Exploração Visual das Correspondências de Cândido Portinari através de Metadados Textuais**
#### **por Micaele Brandão e Mateus Bandeira**

### **Tópicos**
1. Introdução
2. Configuração de ambiente
3. Coleta e limpeza de dados  
.  
.  
.  
6. Conclusão e Referências


# 1. Introdução

Este projeto se trata de uma exploração visual das corresponências de Cândido Portinari, um dos mais importantes pintores brasileiros de todos os tempos. Através do processamento de metadados textuais , buscamos ... , .... e .... . Para isso, utilizamos técnicas de web scraping com o objetivo de transformar documentos históricos em insights visuais que facilitem a compreensão da trajetória pessoal e profissional do artista.

# 2. Configuração de ambiente

In [60]:
import pandas as pd

#scrapping 1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

#scrapping 2
import asyncio
import random
import pandas as pd
from playwright.async_api import async_playwright
from tqdm.notebook import tqdm

import re
import warnings
# configuração opcional para suprimir avisos de concatenação futura, se desejar
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Bibliotecas carregadas")

Bibliotecas carregadas


# 3. Coleta e limpeza de dados

# 3.1. Coleta dos dados
Códigos para fazer o web scrapping do site com as correspondências de Portinari

## 3.2. Definição de Funções Auxiliares
Para manter o código limpo, definimos aqui as funções que serão reutilizadas ao longo do notebook:
1. `processar_e_juntar`: Automatiza a leitura, padronização e concatenação dos arquivos csv.
2. `extrair_campo`: Aplica expressões regulares para extrair metadados do texto bruto.

In [61]:
def processar_e_juntar(df_acumulado, arquivo_novo):
    """
    lê um novo arquivo csv, padroniza as colunas e junta com o dataframe acumulado
    """
    print(f"\n-- Processando junção com: {arquivo_novo} --")
    
    # carregar os arquivos
    try:
        # 'on_bad_lines' pra pular linhas com erro se houver, e encoding utf-8
        df_extraidos = pd.read_csv(arquivo_novo)
    except FileNotFoundError:
        print(f"Erro: Arquivo {arquivo_novo} não encontrado.")
        return df_acumulado

    # ajustar 'dados_extraidos' 
    # o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
    cols_uteis = ['titulo', 'conteudo']
    # Verificação simples se as colunas existem
    if not all(col in df_extraidos.columns for col in cols_uteis):
        print(f"Colunas necessárias ausentes em {arquivo_novo}. Pulando.")
        return df_acumulado
        
    df_extraidos = df_extraidos[cols_uteis].copy()

    # adc as colunas que faltam para ficar no padrão do docvirt
    # se não houver informação, deixamos como None ou um texto padrão
    df_extraidos['pagina'] = None 
    df_extraidos['status'] = 'extraido_novo' 

    # reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
    colunas_ordem = df_acumulado.columns.tolist()
    for col in colunas_ordem:
        if col not in df_extraidos.columns:
            df_extraidos[col] = None
    df_extraidos = df_extraidos[colunas_ordem]

    # juntar e completar
    # concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo 
    df_final = pd.concat([df_acumulado, df_extraidos], ignore_index=True)

    # remover duplicatas
    # se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
    # isso garante que completamos a lista sem criar linhas repetidas
    df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')
    
    print(f"Total de linhas após juntar: {len(df_final)}")
    return df_final

def extrair_campo(texto, padrao):
    """ função para extrair padrões do texto cru usando regex """
    if pd.isna(texto):
        return None
    match = re.search(padrao, texto, re.MULTILINE | re.IGNORECASE)
    return match.group(1).strip() if match else None

## 3.3. Consolidação dos Dados
Carregamos a base principal e iteramos sobre os arquivos novos para criar um único dataframe consolidado.

In [62]:
# arquivo base original
arquivo_base = 'dados_docvirt.csv'
df_final = pd.read_csv(arquivo_base)
print(f"Base inicial carregada ({arquivo_base}). Linhas: {len(df_final)}")

# lista de arquivos para adicionar sequencialmente
arquivos_novos = [
    'dados_extraidos.csv',
    'dados_extraidos2.csv',
    'dados_extraidos3.csv'
]

# loop de processamento
for arquivo in arquivos_novos:
    df_final = processar_e_juntar(df_final, arquivo)

# salva um checkpoint consolidado 
df_final.to_csv('dados_consolidados_bruto.csv', index=False)
print("Consolidação concluída")
display(df_final.head())

Base inicial carregada (dados_docvirt.csv). Linhas: 9126

-- Processando junção com: dados_extraidos.csv --
Total de linhas após juntar: 5803

-- Processando junção com: dados_extraidos2.csv --
Total de linhas após juntar: 5803

-- Processando junção com: dados_extraidos3.csv --
Total de linhas após juntar: 5836
Consolidação concluída


,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\r\nNUM_ANTDOC: 1\r\nPAGINAÇÃO: \r...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\r\nNUM_ANTDOC: 2\r\nPAGINAÇÃO: \r...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\r\nNUM_ANTDOC: 3\r\nPAGINAÇÃO: \r...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\r\nNUM_ANTDOC: 4\r\nPAGINAÇÃO: \r...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\r\nNUM_ANTDOC: 5\r\nPAGINAÇÃO: \r...,sucesso


## 3.4. Tratamento e Padronização
Realizamos a limpeza básica nos tipos de dados e strings:
1. Conversão da coluna `pagina` para numérico;
2. Remoção de espaços em branco;
3. Criação da coluna `id_doc` limpa.

In [63]:
# zjustar a coluna 'pagina' para números inteiros
df_final['pagina'] = pd.to_numeric(df_final['pagina'], errors='coerce').astype('Int64')

# limpeza de espaços em branco em todas as colunas de texto
cols_texto = df_final.select_dtypes(include=['object']).columns
for col in cols_texto:
    df_final[col] = df_final[col].astype(str).str.strip()
    # corrige onde virou string 'nan' ou 'None' para valor nulo real
    df_final[col] = df_final[col].replace({'nan': None, 'None': None, '': None})

# extrair um ID limpo do título
# o título atual é longo e vamos criar uma coluna 'id_doc'
# regex pega tudo que vem depois da última barra invertida
df_final['id_doc'] = df_final['titulo'].astype(str).str.extract(r'\\([^\\]+)$')

print("Tratamento concluído")
display(df_final[['id_doc', 'pagina', 'titulo']].head())

Tratamento concluído


,id_doc,pagina,titulo
0,CO_0001,1,(CO) Consulta Correspondência\CO_0001
1,CO_0002,2,(CO) Consulta Correspondência\CO_0002
2,CO_0003,3,(CO) Consulta Correspondência\CO_0003
3,CO_0004,4,(CO) Consulta Correspondência\CO_0004
4,CO_0005,5,(CO) Consulta Correspondência\CO_0005


## 3.5. Extração de Metadados 
Usamos as expressões regulares definidas para estruturar o texto da coluna `conteudo` em novas colunas (Remetente, Destinatário, Data, Tipo).

In [64]:
# padrões de regex baseados no formato do seu arquivo
padroes = {
    'remetente': r'REMETENTE:\s*(.*?)(?:\n|$)',
    'destinatario': r'DESTINATARIO:\s*(.*?)(?:\n|$)',
    'data': r'DATA_INICIO:\s*(.*?)(?:\n|$)',
    'tipo_doc': r'TIPO CORRESPONDÊNCIA:\s*(.*?)(?:\n|$)'
}

# aplica a extração para criar novas colunas usando a função definida no passo 2
print("Extraindo metadados do conteúdo")

for nova_coluna, regex in padroes.items():
    df_final[nova_coluna] = df_final['conteudo'].apply(lambda x: extrair_campo(str(x), regex))

# visualizar o resultado das novas colunas
colunas_visualizar = ['id_doc'] + list(padroes.keys())
display(df_final[colunas_visualizar].head())

Extraindo metadados do conteúdo


,id_doc,remetente,destinatario,data,tipo_doc
0,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama
1,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta
2,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta
3,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta
4,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta


## 3.6. Auditoria de Dados
Analisamos o que está faltando:
1. Documentos com conteúdo vazio (para busca manual).
2. IDs sequenciais que estão faltando na base (Gap Analysis).

In [65]:
# -- Análise dos vazios -- 
# identificar onde o conteúdo está vazio ou nulo
mask_sem_conteudo = (
    df_final['conteudo'].isna() | 
    (df_final['conteudo'] == '')
)
print(f"Total de documentos com conteúdo vazio (para preenchimento manual): {mask_sem_conteudo.sum()}")

# para exportar os manuais:
# df_manual = df_final[mask_sem_conteudo].copy()
# df_manual.to_csv('para_preencher_manual.csv', index=False)


# -- Análise dos IDs faltantes --
# ver quais são as correspondências que estão faltando entre CO_0001 e CO_5892
ids_esperados = {f'CO_{str(i).zfill(4)}' for i in range(1, 5893)}
ids_presentes = set(df_final['id_doc'].dropna().unique())
ids_faltando = ids_esperados - ids_presentes

print(f"Total de correspondências esperadas: {len(ids_esperados)}")
print(f"Total de correspondências presentes: {len(ids_presentes)}")
print(f"Total de correspondências faltando: {len(ids_faltando)}")

if len(ids_faltando) > 0:
    print("IDs faltando:", list(sorted(ids_faltando)))

Total de documentos com conteúdo vazio (para preenchimento manual): 1672
Total de correspondências esperadas: 5892
Total de correspondências presentes: 5835
Total de correspondências faltando: 73
IDs faltando: ['CO_1807', 'CO_1875', 'CO_2043', 'CO_2095', 'CO_2192', 'CO_2769', 'CO_3027', 'CO_3028', 'CO_3029', 'CO_3037', 'CO_3061', 'CO_3062', 'CO_3066', 'CO_3083', 'CO_3109', 'CO_3123', 'CO_3125', 'CO_3136', 'CO_3164', 'CO_3165', 'CO_3166', 'CO_3167', 'CO_3185', 'CO_3208', 'CO_3439', 'CO_3506', 'CO_3629', 'CO_3674', 'CO_3890', 'CO_4043', 'CO_4047', 'CO_4048', 'CO_4050', 'CO_4051', 'CO_4053', 'CO_4057', 'CO_4063', 'CO_4064', 'CO_4087', 'CO_4098', 'CO_4099', 'CO_4101', 'CO_4116', 'CO_4251', 'CO_4306', 'CO_4307', 'CO_4308', 'CO_4310', 'CO_4311', 'CO_4312', 'CO_4313', 'CO_4320', 'CO_4328', 'CO_4344', 'CO_4353', 'CO_4394', 'CO_4395', 'CO_4405', 'CO_4406', 'CO_4409', 'CO_4412', 'CO_4416', 'CO_4420', 'CO_4681', 'CO_4686', 'CO_4689', 'CO_4690', 'CO_5190', 'CO_5308', 'CO_5318', 'CO_5431', 'CO_5539

## 3.7. Limpeza Final e Salvamento
Realizamos os últimos filtros e ajustes antes de salvar o arquivo final:
1. Filtrar apenas documentos do tipo `CO` (removendo `AP` ou outros).
2. Ordenar pelo ID.
3. Preencher páginas vazias usando a lógica sequencial (`ffill`).

In [66]:
# filtrando os códigos para ter só os CO_NNNNN e tirar os AP_NN.N
# também filtrando os vazios 
df_final = df_final[df_final['conteudo'].fillna('').str.startswith('COD_TIPO: CO')]

print(f"Total de linhas após filtro de tipo CO: {len(df_final)}")

# ordena o df por CO_NNNN crescentemente usando a coluna id_doc 
df_final = df_final.sort_values(by='id_doc')

# completa a coluna página, substituindo os <NA> pelo valor correto ordenado, de acordo com a posição da linha no df
# o ffill propaga o último valor válido para frente, como ordenamos por ID, isso preenche páginas sequenciais.
df_final['pagina'] = df_final['pagina'].ffill()

# teste com o CO_0502, que estava com página vazia no seu exemplo original
print("Verificação do CO_0502:")
display(df_final[df_final['id_doc'] == 'CO_0502'])

# salvar como dados finais
nome_arquivo_final = 'dados_finais.csv'
df_final.to_csv(nome_arquivo_final, index=False)
print(f"Arquivo '{nome_arquivo_final}' salvo com sucesso!")

Total de linhas após filtro de tipo CO: 4147
Verificação do CO_0502:


,pagina,titulo,conteudo,status,id_doc,remetente,destinatario,data,tipo_doc
5805,803,(CO) Consulta Correspondência\CO_0502,COD_TIPO: CO\nNUM_ANTDOC: 502\nPAGINAÇÃO: \nTI...,extraido_novo,CO_0502,Mário Dionísio,Candido Portinari,1952/11/27,Carta


Arquivo 'dados_finais.csv' salvo com sucesso!


---
### ___EXTRA___
Confirmação se a saida de dados desse resumo está igual ao resultado anterior de várias etapas

In [67]:
# teste para saber se dados_finais.csv e dados_finais_temp.csv estão iguais
df_1 = pd.read_csv('dados_finais.csv')

# apenas considerando as 4 primeiras colunas de df_1 para comparação, já que o df_final tem mais colunas do que o df_temp
colunas_comparacao = df_1.columns[:4]  # ajusta para as 4 primeiras colunas
df_1_comparacao = df_1[colunas_comparacao]

df_2 = pd.read_csv('dados_finais_temp.csv')

if df_1_comparacao.equals(df_2[colunas_comparacao]):
    print("Teste de integridade: Os arquivos são idênticos.")
else:
    print("Teste de integridade: Os arquivos são diferentes. Verifique as diferenças.")

Teste de integridade: Os arquivos são diferentes. Verifique as diferenças.


In [68]:
# Exibir as diferenças entre os arquivos, se houver
if not df_1_comparacao.equals(df_2[colunas_comparacao]):
    # Encontrar as diferenças usando merge com indicador
    df_diff = df_1_comparacao.merge(df_2[colunas_comparacao], indicator=True, how='outer')
    
    # Linhas que estão apenas em df_1
    apenas_df_1 = df_diff[df_diff['_merge'] == 'left_only']
    print("\nLinhas presentes apenas em dados_finais.csv:")
    display(apenas_df_1)
    
    # Linhas que estão apenas em df_2
    apenas_df_2 = df_diff[df_diff['_merge'] == 'right_only']
    print("\nLinhas presentes apenas em dados_finais_temp.csv:")
    display(apenas_df_2)
else:
    print("Nenhuma diferença encontrada entre os arquivos nas colunas comparadas.")


Linhas presentes apenas em dados_finais.csv:


,pagina,titulo,conteudo,status,_merge
501,803,(CO) Consulta Correspondência\CO_0502,COD_TIPO: CO\nNUM_ANTDOC: 502\nPAGINAÇÃO: \nTI...,extraido_novo,left_only
560,882,(CO) Consulta Correspondência\CO_0560,COD_TIPO: CO\nNUM_ANTDOC: 560\nPAGINAÇÃO: \nTI...,extraido_novo,left_only
620,964,(CO) Consulta Correspondência\CO_0619,COD_TIPO: CO\nNUM_ANTDOC: 619\nPAGINAÇÃO: \nTI...,extraido_novo,left_only
709,1091,(CO) Consulta Correspondência\CO_0707,COD_TIPO: CO\nNUM_ANTDOC: 707\nPAGINAÇÃO: 2 p....,extraido_novo,left_only
852,1302,(CO) Consulta Correspondência\CO_0849,COD_TIPO: CO\nNUM_ANTDOC: 849\nPAGINAÇÃO: \nTI...,extraido_novo,left_only
1008,1514,(CO) Consulta Correspondência\CO_1004,COD_TIPO: CO\nNUM_ANTDOC: 1004\nPAGINAÇÃO: \nT...,extraido_novo,left_only
1750,2715,(CO) Consulta Correspondência\CO_1745,COD_TIPO: CO\nNUM_ANTDOC: 1745\nPAGINAÇÃO: 2 p...,extraido_novo,left_only
1799,2809,(CO) Consulta Correspondência\CO_1793,COD_TIPO: CO\nNUM_ANTDOC: 1793\nPAGINAÇÃO: \nT...,extraido_novo,left_only
1805,2814,(CO) Consulta Correspondência\CO_1798,COD_TIPO: CO\nNUM_ANTDOC: 1798\nPAGINAÇÃO: \nT...,extraido_novo,left_only
1825,2836,(CO) Consulta Correspondência\CO_1818,COD_TIPO: CO\nNUM_ANTDOC: 1818\nPAGINAÇÃO: 2 p...,extraido_novo,left_only



Linhas presentes apenas em dados_finais_temp.csv:


,pagina,titulo,conteudo,status,_merge
502,803,(CO) Consulta Correspondência\CO_0502,COD_TIPO: CO\r\nNUM_ANTDOC: 502\r\nPAGINAÇÃO: ...,extraido_novo,right_only
561,882,(CO) Consulta Correspondência\CO_0560,COD_TIPO: CO\r\nNUM_ANTDOC: 560\r\nPAGINAÇÃO: ...,extraido_novo,right_only
621,964,(CO) Consulta Correspondência\CO_0619,COD_TIPO: CO\r\nNUM_ANTDOC: 619\r\nPAGINAÇÃO: ...,extraido_novo,right_only
710,1091,(CO) Consulta Correspondência\CO_0707,COD_TIPO: CO\r\nNUM_ANTDOC: 707\r\nPAGINAÇÃO: ...,extraido_novo,right_only
853,1302,(CO) Consulta Correspondência\CO_0849,COD_TIPO: CO\r\nNUM_ANTDOC: 849\r\nPAGINAÇÃO: ...,extraido_novo,right_only
1009,1514,(CO) Consulta Correspondência\CO_1004,COD_TIPO: CO\r\nNUM_ANTDOC: 1004\r\nPAGINAÇÃO:...,extraido_novo,right_only
1751,2715,(CO) Consulta Correspondência\CO_1745,COD_TIPO: CO\r\nNUM_ANTDOC: 1745\r\nPAGINAÇÃO:...,extraido_novo,right_only
1800,2809,(CO) Consulta Correspondência\CO_1793,COD_TIPO: CO\r\nNUM_ANTDOC: 1793\r\nPAGINAÇÃO:...,extraido_novo,right_only
1806,2814,(CO) Consulta Correspondência\CO_1798,COD_TIPO: CO\r\nNUM_ANTDOC: 1798\r\nPAGINAÇÃO:...,extraido_novo,right_only
1826,2836,(CO) Consulta Correspondência\CO_1818,COD_TIPO: CO\r\nNUM_ANTDOC: 1818\r\nPAGINAÇÃO:...,extraido_novo,right_only


___

## 4. Trabalhando com o df

In [69]:
df = pd.read_csv('dados_finais.csv')
df.head(50)

,pagina,titulo,conteudo,status,id_doc,remetente,destinatario,data,tipo_doc
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\r\nNUM_ANTDOC: 1\r\nPAGINAÇÃO: \r...,sucesso,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\r\nNUM_ANTDOC: 2\r\nPAGINAÇÃO: \r...,sucesso,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\r\nNUM_ANTDOC: 3\r\nPAGINAÇÃO: \r...,sucesso,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\r\nNUM_ANTDOC: 4\r\nPAGINAÇÃO: \r...,sucesso,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\r\nNUM_ANTDOC: 5\r\nPAGINAÇÃO: \r...,sucesso,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta
5,6,(CO) Consulta Correspondência\CO_0006,COD_TIPO: CO\r\nNUM_ANTDOC: 6\r\nPAGINAÇÃO: \r...,sucesso,CO_0006,Manoel de Abreu,Candido Portinari,1950/08/23,Bilhete
6,7,(CO) Consulta Correspondência\CO_0007,COD_TIPO: CO\r\nNUM_ANTDOC: 7\r\nPAGINAÇÃO: \r...,sucesso,CO_0007,Stahlembrecher,Candido Portinari,1928/08/27,Bilhete
7,9,(CO) Consulta Correspondência\CO_0008,COD_TIPO: CO\r\nNUM_ANTDOC: 8\r\nPAGINAÇÃO: \r...,sucesso,CO_0008,Olegário Mariano,Baptista Portinari,1928/ /,Telegrama
8,10,(CO) Consulta Correspondência\CO_0009,COD_TIPO: CO\r\nNUM_ANTDOC: 9\r\nPAGINAÇÃO: 3 ...,sucesso,CO_0009,Ronald de Carvalho,Raul Tavares,1929/06/27,Carta
9,13,(CO) Consulta Correspondência\CO_0010,COD_TIPO: CO\r\nNUM_ANTDOC: 10\r\nPAGINAÇÃO: \...,sucesso,CO_0010,Rose Marie Cimonier,Candido Portinari,1930/02/05,Carta


In [70]:
print("Colunas do df final:")
print(df.columns.tolist())

# info
print("\nInformações do df:")
df.info()

# describe
'''print("\nDescrição estatística do df:")
df.describe(include='all')'''

Colunas do df final:
['pagina', 'titulo', 'conteudo', 'status', 'id_doc', 'remetente', 'destinatario', 'data', 'tipo_doc']

Informações do df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4147 entries, 0 to 4146
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pagina        4147 non-null   int64 
 1   titulo        4147 non-null   object
 2   conteudo      4147 non-null   object
 3   status        4147 non-null   object
 4   id_doc        4147 non-null   object
 5   remetente     4017 non-null   object
 6   destinatario  4094 non-null   object
 7   data          4140 non-null   object
 8   tipo_doc      4147 non-null   object
dtypes: int64(1), object(8)
memory usage: 291.7+ KB


'print("\nDescrição estatística do df:")\ndf.describe(include=\'all\')'

In [71]:
# verificar onde existem valores nulos ou vazios
print("\nContagem de valores nulos por coluna:")
print(df.isnull().sum())


Contagem de valores nulos por coluna:
pagina            0
titulo            0
conteudo          0
status            0
id_doc            0
remetente       130
destinatario     53
data              7
tipo_doc          0
dtype: int64


# X. Limpeza dos dados